<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=03c4cbaf71e3fb762a6b56da8649bec1837e5f4fe03885de8654ab6002b53121
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-10 10:42:38
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: 29.72 K (0.21%)
Current PnL: -21.48 L (-14.08%)
CY Booked + Current PnL: -8.83 L (-5.79%)
-------------------
Total profit:  2.15 L
Total loss:  -23.64 L
-------------------
Total Booked + Current PnL: 18.15 L (14.45%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.84%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.46 L (61.82%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,63.0,X-LC,6.74,232565.0,22199.0,6582.0,0.87,10.55,2.83,13.68,37.0,3.37,1.64,28.82,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,67.0,M-LC,11.49,173711.0,32871.0,11621.0,-0.34,23.34,6.69,31.59,66.0,2.83,1.22,56.01,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,101.28,51.0,M-SC,2.01,86676.0,-14101.0,14198.0,-1.00,-13.99,16.38,0.09,245.0,-0.99,0.61,13.87,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-13.14,59.0,X-MC,6.25,149362.0,13229.0,17356.0,-0.09,9.72,11.62,22.47,91.0,0.76,1.05,23.96,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-11.87,38.0,X-MC,3.11,87075.0,-3603.0,18512.0,0.83,-3.97,21.26,16.44,101.0,-0.19,0.61,15.40,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIGOPNTS,1408.00,108.71,66.0,M-SC,6.03,137008.0,-37551.0,37581.0,10.04,-21.51,27.43,0.02,221.0,-1.00,0.96,19.17,OX40N,NTT,PAINTS
39,INFY,2275.00,-17.44,60.0,X-LC,4.87,321779.0,8891.0,162788.0,2.30,2.84,50.59,54.87,3.0,0.05,2.27,11.36,X40,BTT,IT
31,HINDZINC,730.22,29.16,53.0,M-LC,9.40,210670.0,5594.0,106978.0,2.07,2.73,50.78,54.89,52.0,0.05,1.48,27.48,X5K,ATH,METALS
6,ASIANPAINT,3465.66,-7.05,75.0,X-LC,5.64,234520.0,-17248.0,70450.0,1.96,-6.85,30.04,21.13,27.0,-0.24,1.65,26.15,X40,ATH,PAINTS
29,HCLTECH,1922.01,5.81,61.0,X-LC,5.21,241953.0,29.0,59811.0,1.90,0.01,24.72,24.73,8.0,0.00,1.70,15.25,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-194.32,59.0,M-SC,34.66,162324.0,13104.0,131077.0,-2.72,8.78,80.75,96.62,208.0,0.10,1.14,75.67,XY25,NTT,FINANCE
85,WHIRLPOOL,2270.00,-42.05,47.0,M-SC,7.06,120216.0,5418.0,88623.0,-2.49,4.72,73.72,81.92,223.0,0.06,0.85,42.32,XR,NTT,DURABLES
25,GREENPANEL,537.00,204.91,41.0,M-SC,5.27,136103.0,-46975.0,125950.0,-2.23,-25.66,92.54,43.14,230.0,-0.37,0.96,28.02,XY24,NTT,MISC
70,SURYODAY,240.00,55.70,41.0,H-SC,14.66,143157.0,-35914.0,101641.0,-2.04,-20.06,71.00,36.71,135.0,-0.35,1.01,42.04,XR,NTT,BANKS
46,KPIGREEN,731.05,4.41,47.0,H-SC,9.50,124425.0,-872.0,59799.0,-1.88,-0.70,48.06,47.03,141.0,-0.01,0.88,55.79,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,4.41,47.0,H-SC,9.5,124425.0,-872.0,59799.0,-1.88,-0.7,48.06,47.03,141.0,-0.01,0.88,55.79,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.28,51.0,M-SC,2.01,86676.0,-14101.0,14198.0,-1.00,-13.99,16.38,0.09,245.0,-0.99,0.61,13.87,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,42.0,H-SC,2.32,220311.0,-49356.0,85679.0,-0.62,-18.30,38.89,13.47,138.0,-0.58,1.55,12.02,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,108.71,66.0,M-SC,6.03,137008.0,-37551.0,37581.0,10.04,-21.51,27.43,0.02,221.0,-1.00,0.96,19.17,OX40N,NTT,PAINTS
67,SIS,528.0,2037.31,49.0,H-SC,2.31,86254.0,-24778.0,48388.0,1.70,-22.32,56.10,21.26,156.0,-0.51,0.61,16.54,OX40N,NTT,MISC
17,CERA,9475.0,-25.06,39.0,H-SC,1.86,136436.0,-39467.0,81493.0,-1.08,-22.44,59.73,23.89,149.0,-0.48,0.96,17.83,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,67.0,M-LC,11.49,173711.0,32871.0,11621.0,-0.34,23.34,6.69,31.59,66.0,2.83,1.22,56.01,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,54.77,67.0,M-MC,9.92,244817.0,19855.0,147796.0,0.83,8.83,60.37,74.53,192.0,0.13,1.72,43.93,XY24,BTT,STEEL
31,HINDZINC,730.22,29.16,53.0,M-LC,9.40,210670.0,5594.0,106978.0,2.07,2.73,50.78,54.89,52.0,0.05,1.48,27.48,X5K,ATH,METALS
79,UNIONBANK,163.00,-3.88,67.0,M-LC,11.49,173711.0,32871.0,11621.0,-0.34,23.34,6.69,31.59,66.0,2.83,1.22,56.01,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-42.05,47.0,M-SC,7.06,120216.0,5418.0,88623.0,-2.49,4.72,73.72,81.92,223.0,0.06,0.85,42.32,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,50.0,H-SC,5.24,129108.0,5772.0,125854.0,-0.47,4.68,97.48,106.72,119.0,0.05,0.91,29.15,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-42.05,47.0,M-SC,7.06,120216.0,5418.0,88623.0,-2.49,4.72,73.72,81.92,223.0,0.06,0.85,42.32,XR,NTT,DURABLES
31,HINDZINC,730.22,29.16,53.0,M-LC,9.40,210670.0,5594.0,106978.0,2.07,2.73,50.78,54.89,52.0,0.05,1.48,27.48,X5K,ATH,METALS
36,INDIAMART,4810.62,-53.71,50.0,H-SC,5.24,129108.0,5772.0,125854.0,-0.47,4.68,97.48,106.72,119.0,0.05,0.91,29.15,AR,ATH,MISC
46,KPIGREEN,731.05,4.41,47.0,H-SC,9.50,124425.0,-872.0,59799.0,-1.88,-0.70,48.06,47.03,141.0,-0.01,0.88,55.79,MH,ATH,POWER
86,WIPRO,420.00,-15.61,44.0,M-LC,5.61,148428.0,-2517.0,111974.0,1.23,-1.67,75.44,72.51,53.0,-0.02,1.05,4.25,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,34.23,147269.0,-129254.0,239327.0,-0.53,-46.74,162.51,39.81,54.0,-0.54,1.04,3.80,XY24,NTT,AUTO
23,DMART,5391.45,-20.99,27.0,X-LC,7.17,201300.0,-9107.0,68281.0,0.37,-4.33,33.92,28.12,38.0,-0.13,1.42,18.27,X40N,ATH,FMCG
12,BATAINDIA,2096.00,-41.44,28.0,X-SC,8.13,87474.0,-41196.0,86494.0,-0.75,-32.02,98.88,35.20,219.0,-0.48,0.62,0.40,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-23.55,32.0,X-MC,4.45,191700.0,-8647.0,39145.0,1.36,-4.32,20.42,15.23,178.0,-0.22,1.35,26.54,X40N,NTT,REALTY
1,ABB,7934.00,-42.69,33.0,H-LC,4.95,239520.0,-22099.0,141317.0,-0.53,-8.45,59.00,45.57,7.0,-0.16,1.69,1.92,AR,NTT,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.34,37.0,X-MC,0.28,198675.0,-2145.0,69934.0,0.54,-1.07,35.20,33.76,80.0,-0.03,1.40,4.07,X40,ATH,FMCG
41,ITC,452.00,-38.34,45.0,X-LC,0.81,198827.0,-1311.0,22209.0,0.63,-0.66,11.17,10.44,4.0,-0.06,1.40,4.87,X40,NTT,FMCG
66,SIEMENS,4671.50,-6.25,35.0,H-LC,1.60,150720.0,-35375.0,82851.0,-0.63,-19.01,54.97,25.51,15.0,-0.43,1.06,11.07,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.02,41.0,X-MC,3.27,306852.0,-21139.0,127681.0,0.88,-6.44,41.61,32.48,92.0,-0.17,2.16,4.24,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.81,62.0,X-LC,3.79,244446.0,10302.0,38329.0,-0.55,4.40,15.68,20.77,86.0,0.27,1.72,11.62,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.44,28.0,X-SC,8.13,87474.0,-41196.0,86494.0,-0.75,-32.02,98.88,35.20,219.0,-0.48,0.62,0.40,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.24,36.0,X-MC,7.55,79880.0,-3630.0,23660.0,0.50,-4.35,29.62,23.98,82.0,-0.15,0.56,1.22,X40,ATH,APPARELS
53,QUESS,986.00,-49.86,37.0,X-SC,37.30,49693.0,-15313.0,166238.0,0.16,-23.56,334.53,232.18,198.0,-0.09,0.35,1.70,XY24,NTT,MISC
1,ABB,7934.00,-42.69,33.0,H-LC,4.95,239520.0,-22099.0,141317.0,-0.53,-8.45,59.00,45.57,7.0,-0.16,1.69,1.92,AR,NTT,ELECTRICAL
20,COLPAL,3726.84,-6.74,41.0,X-MC,7.18,217600.0,-45765.0,155084.0,0.41,-17.38,71.27,41.51,84.0,-0.30,1.53,2.11,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.86,37.0,X-SC,37.30,49693.0,-15313.0,166238.0,0.16,-23.56,334.53,232.18,198.0,-0.09,0.35,1.70,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.24,36.0,X-MC,7.55,79880.0,-3630.0,23660.0,0.50,-4.35,29.62,23.98,82.0,-0.15,0.56,1.22,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.65,41.0,X-SC,4.20,80868.0,-64292.0,140217.0,-0.30,-44.29,173.39,52.31,136.0,-0.46,0.57,8.25,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.87,38.0,X-MC,3.11,87075.0,-3603.0,18512.0,0.83,-3.97,21.26,16.44,101.0,-0.19,0.61,15.40,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.44,28.0,X-SC,8.13,87474.0,-41196.0,86494.0,-0.75,-32.02,98.88,35.20,219.0,-0.48,0.62,0.40,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.36,52.0,X-LC,9.54,285403.0,-60553.0,127261.0,1.48,-17.50,44.59,19.28,1.0,-0.48,2.01,5.51,X40,ATH,IT
39,INFY,2275.00,-17.44,60.0,X-LC,4.87,321779.0,8891.0,162788.0,2.30,2.84,50.59,54.87,3.0,0.05,2.27,11.36,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,0.81,198827.0,-1311.0,22209.0,0.63,-0.66,11.17,10.44,4.0,-0.06,1.40,4.87,X40,NTT,FMCG
83,VBL,671.64,-17.33,51.0,X-LC,5.65,297786.0,-18056.0,130728.0,-0.73,-5.72,43.90,35.67,5.0,-0.14,2.10,7.35,X40N,ATH,FMCG
1,ABB,7934.00,-42.69,33.0,H-LC,4.95,239520.0,-22099.0,141317.0,-0.53,-8.45,59.00,45.57,7.0,-0.16,1.69,1.92,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,33.0,L-SC,27.26,73958.0,-39591.0,79638.0,-1.21,-34.87,107.68,35.27,268.0,-0.50,0.52,81.02,XR,NTT,HOTELS
7,ASIANTILES,137.00,7294.44,60.0,L-SC,12.93,81134.0,-12676.0,89296.0,-0.78,-13.51,110.06,81.67,269.0,-0.14,0.57,57.12,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,50.0,H-SC,7.97,92130.0,-5850.0,27455.0,-0.08,-5.97,29.80,22.05,152.0,-0.21,0.65,33.92,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-42.05,47.0,M-SC,7.06,120216.0,5418.0,88623.0,-2.49,4.72,73.72,81.92,223.0,0.06,0.85,42.32,XR,NTT,DURABLES
46,KPIGREEN,731.05,4.41,47.0,H-SC,9.50,124425.0,-872.0,59799.0,-1.88,-0.70,48.06,47.03,141.0,-0.01,0.88,55.79,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,71.90,62.0,H-SC,4.84,148981.0,-33794.0,144869.0,-0.64,-18.49,97.24,60.77,125.0,-0.23,1.05,34.17,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,-7.05,75.0,X-LC,5.64,234520.0,-17248.0,70450.0,1.96,-6.85,30.04,21.13,27.0,-0.24,1.65,26.15,X40,ATH,PAINTS
5,ANGELONE,3033.00,20.92,66.0,X-SC,6.42,213790.0,22784.0,34912.0,-0.35,11.93,16.33,30.21,157.0,0.65,1.51,35.30,X40N,NTT,FINANCE
68,SONACOMS,806.63,-31.54,65.0,M-SC,6.27,85190.0,-15969.0,55970.0,-0.88,-15.79,65.70,39.54,202.0,-0.29,0.60,20.13,AR,ATH,AUTO
85,WHIRLPOOL,2270.00,-42.05,47.0,M-SC,7.06,120216.0,5418.0,88623.0,-2.49,4.72,73.72,81.92,223.0,0.06,0.85,42.32,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.80
1,25,44.53
2,50,75.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.71
LC    33.25
MC    23.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.94
X40      16.41
X40N     12.48
XY25     11.92
XR        9.80
AR        7.76
OX40N     6.56
X200      1.79
X5K       1.48
SR        1.02
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.12
X-LC    22.56
X-MC    18.72
M-SC    12.57
X-SC     5.60
M-LC     5.16
H-LC     4.54
H-MC     2.32
M-MC     1.72
L-SC     1.42
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.22,-4.73,38.59
IT,12.58,-17.76,80.79
FINANCE,12.15,-9.54,59.20
MISC,7.77,-17.79,77.20
PAINTS,5.68,-14.38,31.43
ELECTRICAL,5.68,-12.15,51.91
BANKS,4.00,-18.95,88.15
INSURANCE,3.92,0.75,36.25
AUTO,2.70,-49.90,112.47


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3322700.0,22
XR,1266601.0,13
AR,1186886.0,9
X40,825463.0,12
X40N,698157.0,9
OX40N,552647.0,9
XY25,482024.0,8
SR,275339.0,2
X5K,106978.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3406946.0,24
M-SC,1260886.0,15
X-MC,1214378.0,14
X-LC,1076098.0,13
X-SC,556332.0,6
H-LC,293329.0,3
M-LC,276037.0,4
L-SC,258997.0,3
H-MC,258842.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1189325.0      6
           AR         838763.0      5
           XR         790723.0      7
M-SC       XY24       733269.0      6
X-MC       XY24       577159.0      4
X-LC       X40        442519.0      5
           XY24       313428.0      2
X-SC       X40N       303600.0      4
X-MC       X40        296450.0      6
H-SC       SR         275339.0      2
           OX40N      252997.0      3
X-LC       X40N       237338.0      3
H-LC       AR         224168.0      2
H-MC       XY24       183864.0      1
X-MC       XY25       183550.0      2
L-SC       XR         168934.0      2
X-SC       XY24       166238.0      1
X-MC       X40N       157219.0      2
M-MC       XY24       147796.0      1
M-SC       XR         137976.0      2
           OX40N      134609.0      4
           XY25       131077.0      1
           AR         123955.0      2
M-LC       XR         111974.0      1
           X5K        106978.0      1
L-SC       OX40N       90063.0      1
X-SC       X40         86494.0      1
X-LC       XY25        82813.0      3
H-MC       OX40N       74978.0      1
H-LC       X200        69161.0      1
H-SC       MH          59799.0      1
L-MC       XR          56994.0      1
M-LC       XY25        45464.0      1
L-LC       XY25        39120.0      1
M-LC       XY24        11621.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
